In [2]:
import sqlite3
import os
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
from desispec.coaddition import coadd
from desiutil.log import get_logger, DEBUG
log = get_logger()
log.setLevel(DEBUG)

In [3]:
filename = "/global/cfs/cdirs/desi/science/td/db/desi.db"


In [4]:
con = sqlite3.connect(filename)
cur = con.cursor()

## Note that the following doesn't list multiple exposures of the same tile the same night

In [5]:
targetid_q = """
select distinct mtl.targetid, f.yyyymmdd, f.tile, f.petal
from mtl
inner join fibermap_daily f
on mtl.targetid==f.targetid
where mtl.program='ToO'
order by mtl.targetid, f.yyyymmdd, f.tile, f.petal ;
"""

In [6]:
cur.execute(targetid_q)
q_arr = cur.fetchall()
cur.close()

## Produce list that groups tile/petal combinations observed for the same targetid the same night

In [7]:
ans =[]
tid_night=[q_arr[0]]
for i in range(1,len(q_arr)):
    if q_arr[i][0]== tid_night[0][0] and q_arr[i][1]== tid_night[0][1]:
        tid_night.append(q_arr[i])
    else:
        ans.append(tid_night)
        tid_night=[q_arr[i]]
        if i == len(q_arr)-1:
             ans.append(tid_night)

In [8]:
datapath = os.path.join(os.environ.get('DESI_REDUX'),'daily','tiles','cumulative')

In [ ]:
spectra_out=[]
for a in ans:
    spectra=Spectra()
    for targetid, yyyymmdd, tile, petal in a:
        filename = os.path.join(datapath,str(tile),str(yyyymmdd),f"spectra-{petal}-{tile}-thru{yyyymmdd}.fits")
        spec = read_spectra(filename)
        spectra.update(spec.select(targets=[targetid]))
    if spectra.num_spectra()>1:
        coadd(spectra)
    spectra_out.append(spectra)

43977408033849634 20210504 233 0
INFO:spectra.py:253:read_spectra: iotime 0.780 sec to read spectra-0-233-thru20210504.fits at 2021-05-27T08:27:15.303238
43977408033849634 20210505 234 0
INFO:spectra.py:253:read_spectra: iotime 0.682 sec to read spectra-0-234-thru20210505.fits at 2021-05-27T08:27:16.409140
43977408096764091 20210430 261 0
INFO:spectra.py:253:read_spectra: iotime 1.099 sec to read spectra-0-261-thru20210430.fits at 2021-05-27T08:27:17.931881
43977408096764276 20210502 262 0
INFO:spectra.py:253:read_spectra: iotime 0.695 sec to read spectra-0-262-thru20210502.fits at 2021-05-27T08:27:19.538562
43977408096764276 20210503 262 0
INFO:spectra.py:253:read_spectra: iotime 1.096 sec to read spectra-0-262-thru20210503.fits at 2021-05-27T08:27:21.081922
43977408096764276 20210504 263 0
INFO:spectra.py:253:read_spectra: iotime 0.613 sec to read spectra-0-263-thru20210504.fits at 2021-05-27T08:27:22.510826
43977408096764276 20210505 254 0
INFO:spectra.py:253:read_spectra: iotime 0.